In [113]:
import pandas as pd

from utils.audio_dataset_utils import train_val_dataset
from network_models.soundsream_models_and_utils.ss_model_dim_red import SSDimRedModel
from network_models.soundsream_models_and_utils.ss_model_conv import SSConvModel3Sec
from network_models.soundsream_models_and_utils.ss_encoded_dataset import ss_encoded_dataset_full
import torch
from torch.utils.data import DataLoader

batch_size = 1

data_set= ss_encoded_dataset_full(
    csvPath="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/allEncodings.pkl", device="cuda")

json_path ="/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/"

modelDimRed = SSDimRedModel().cuda()
modelConv = SSConvModel3Sec(xSize=512, ySize=175).cuda()



In [114]:
modelConv.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_conv/emo_reco_best_ep216_acc_62"))
modelDimRed.load_state_dict(torch.load("../content/soundstream/experiments/nr2Run_Nr_0_dimred/emo_reco_500.pth"))

<All keys matched successfully>

In [120]:
from numpy import ndarray


def generateParameters(device, model, dataset, labels):
    loader = DataLoader(dataset, shuffle=False, batch_size=1)
    d ={}

    for label in labels:
        d.update({label: []})

    with torch.no_grad():
        for batch, (X, z) in enumerate(loader):
            X, z = X.to(device),  z.to(device)
            dims, pred = model(X, return_with_dims= True)
            label = data_set.encoded_dataset.label_list[z[0].nonzero().cpu().numpy()[0][0]]
            dims = dims.cpu().numpy()[0]
            d[label].append(dims)
            if batch % 500 == 0:
                print(f"{batch*batch_size} of {len(dataset)}")
    return d


def genAvgFull(fulldata: list[dict], label_list, modelNameList):
    fullDict = {}
    for label in label_list:
        fullDict.update({label: genAvgAndVar([fd[label] for fd in fulldata], modelNameList)})
    return fullDict

def genAvgAndVar(data: list[list[ndarray[float]]], modelNameList):
    d = {}
    for i in range(len(modelNameList)):
        mean = np.mean(np.asarray(data[i]), axis=0)
        var = np.var(np.asarray(data[i]), axis=0)
        d.update({modelNameList[i] : pd.Series([mean, var], index=["mean", "var"])})
    return pd.DataFrame(d)


def generateJson(data):
    str = "{"
    for i in range(len(data)):
        str += f"x{i+1}: {data[i]},"
    str += "}"
    return(str)

def writeJson(emotion, string):
    print(f"writing: {string} \n to {json_path+emotion}.json")
    file = open(json_path+f"{emotion}.json", "w")
    file.write(string)
    file.close()

In [100]:
emo = generateParameters("cuda", modelDimRed, data_set, data_set.encoded_dataset.label_list)
conv_emo = generateParameters("cuda", modelConv, data_set, data_set.encoded_dataset.label_list)


/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092
0 of 6092
500 of 6092
1000 of 6092
1500 of 6092
2000 of 6092
2500 of 6092
3000 of 6092
3500 of 6092
4000 of 6092
4500 of 6092
5000 of 6092
5500 of 6092
6000 of 6092


In [107]:
fullDict = genAvgFull([emo, conv_emo], data_set.encoded_dataset.label_list, ["red", "conv"])


In [108]:
print(f"neutral :  redDim: {fullDict['neutral']['red']['mean']}, conv: {fullDict['neutral']['conv']['mean']}")
print(f"happy   :  redDim: {fullDict['happy']['red']['mean']}, conv: {fullDict['happy']['conv']['mean']}")
print(f"sad     :  redDim: {fullDict['sad']['red']['mean']}, conv: {fullDict['sad']['conv']['mean']}")
print(f"angry   :  redDim: {fullDict['angry']['red']['mean']}, conv: {fullDict['angry']['conv']['mean']}")
print(f"fear    :  redDim: {fullDict['fear']['red']['mean']}, conv: {fullDict['fear']['conv']['mean']}")
print(f"disgust :  redDim: {fullDict['disgust']['red']['mean']}, conv: {fullDict['disgust']['conv']['mean']}")
print(f"surprise:  redDim: {fullDict['surprise']['red']['mean']}, conv: {fullDict['surprise']['conv']['mean']}")



neutral :  redDim: [ 0.6639264   0.73435247 -0.7320692  -0.6289696 ], conv: [-0.4218498 -0.9272894  0.2597164 -0.8949495]
happy   :  redDim: [-0.59133476 -0.6771037  -0.1524691  -0.50330085], conv: [-0.9167148   0.8094782  -0.95139974  0.5421657 ]
sad     :  redDim: [ 0.7622332   0.53079635  0.55524313 -0.6929692 ], conv: [-0.33834997  0.82089335  0.9236266  -0.5644345 ]
angry   :  redDim: [-0.66945684 -0.5150308   0.64851785  0.7231073 ], conv: [ 0.92897815 -0.92863935 -0.9120094   0.38886377]
fear    :  redDim: [0.68669516 0.59777653 0.42156133 0.5944756 ], conv: [ 0.9076029   0.8969184  -0.6527667   0.91186833]
disgust :  redDim: [ 0.60760134 -0.75726825  0.58009577 -0.5529675 ], conv: [-0.8281549  -0.9411225   0.89458096  0.9407132 ]
surprise:  redDim: [ 0.6731572  -0.64274424  0.48773807  0.6123621 ], conv: [ 0.87585086  0.8034662  -0.92787725 -0.92888534]


In [121]:
writeJson("neutral", generateJson(fullDict['neutral']['conv']['mean']))
writeJson("happy", generateJson(fullDict['happy']['conv']['mean']))
writeJson("sad", generateJson(fullDict['sad']['conv']['mean']))
writeJson("angry", generateJson(fullDict['angry']['conv']['mean']))
writeJson("fear", generateJson(fullDict['fear']['conv']['mean']))
writeJson("disgust", generateJson(fullDict['disgust']['conv']['mean']))
writeJson("surprise", generateJson(fullDict['surprise']['conv']['mean']))

writing: {x1: -0.42184978723526,x2: -0.9272894263267517,x3: 0.2597163915634155,x4: -0.8949494957923889,} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/neutral.json
writing: {x1: -0.9167147874832153,x2: 0.809478223323822,x3: -0.9513997435569763,x4: 0.5421656966209412,} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/happy.json
writing: {x1: -0.33834996819496155,x2: 0.8208933472633362,x3: 0.9236266016960144,x4: -0.5644345283508301,} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/sad.json
writing: {x1: 0.9289781451225281,x2: -0.9286393523216248,x3: -0.9120094180107117,x4: 0.38886377215385437,} 
 to /home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/data/angry.json
writing: {x1: 0.9076029062271118,x2: 0.8969184160232544,x3: -0.6527667045593262,x4: 0.9118683338165283,} 
 to /home/ckwdani/Programming/Projects/mas